# Lesson 1: Shared object files

> Disclaimer: we will be working in notebooks, because it's a great place to teach in. It's not a great place to write compiled code in, so we will write out the files we want to compile from the notebooks. In real life, just write out the files directly!

In [ ]:
import sys

# Compilers:
CXX = 'clang++' if 'darwin' in sys.platform else 'gxx'
CC = 'clang' if 'darwin' in sys.platform else 'gcc'

# What is meant by bindings?

Bindings allow a function(alitiy) in a library to be accessed from Python.

#### We will start with this example:

In [ ]:
%%writefile simple.c

float square(float x) {
    return x*x;
}

Depending on your platform, you may normally use gcc (Linux) or clang (macOS) to compile. We'll use `cc` to be compatible with as much as possible. If you are on Windows, you might want to work in the Linux-as-a-Subsystem.

In [ ]:
!cc simple.c -shared -o simple.so

#### Desired usage in Python:

```python
y = square(x)
```

# [ctypes](https://docs.python.org/3.7/library/ctypes.html)

* 0 dependencies
* Shared libraries with C interfaces only
* No help with creating/managing the SO's.

C bindings are very easy. Just compile into a shared library, then open it in Python with the built in [ctypes](https://docs.python.org/3.7/library/ctypes.html) module:

In [ ]:
import ctypes

lib = ctypes.cdll.LoadLibrary('./simple.so')

> Warning: You can only load a file once; future loads just reuse the existing library (much like Python imports). So if you change the file, you need to restart the kernel.

In [ ]:
lib.square.argtypes = (ctypes.c_float,)
lib.square.restype = ctypes.c_float

In [ ]:
# %%timeit
lib.square(2.0)

Notice what was required and why:

1. First we load the library, just handing the file we want to open.
1. Next, we set `.argtypes` (argument types) with a interable of types and `.restype` with the return type. We had to use the types from ctypes since compiled types are not the same as Python types. SO's do not store signatures!

There is an alternate way to do this:

1. Create a new C Function type, listing the return type, then the argument type(s) if there are any. Yes, this is different order and structure than Type Hinting in modren Python...

In [ ]:
float_float_t = ctypes.CFUNCTYPE(ctypes.c_float, ctypes.c_float)

Now "cast" the function with your function type.

In [ ]:
squarefunc = float_float_t(lib.square)

And this also works.

In [ ]:
# %%timeit
squarefunc(2.0)

However, it sets up a *little* bit of extra machinery so it's a hair slower. However, this is itself now a valid c_type and can be passed to code! You can even wrap Python funtions (won't be faster, but powerful, nevertheless)!

Suggestion: wrap your library in a class!

In [ ]:
class SimpleLib(object):
    def __init__(self, file_to_load: str):
        self._lib = ctypes.cdll.LoadLibrary(file_to_load)
        self._lib.square.argtypes = (ctypes.c_float,)
        self._lib.square.restype = ctypes.c_float
        
        
    def square(self, x: float) -> float:
        return self._lib.square(x)

In [ ]:
simple = SimpleLib('./simple.so')
simple.square(2)

> Side note: Yes, I am quite aware you don't need the `(object)` any more in Python 3. But the Python 1 style class you get by forgetting it in Python 2 is so horrible, I don't want to risk it when showing general code.

There is a benefit to these types; they can wrap pure Python functions are fully callable from C, as well!

In [ ]:
%%writefile advanced.cpp

#include <random>

float square(float x) {
    return x*x;
}

### But what about C++ (and all other languages)?

This wasn't really language specific, it is just a property of compiled libraries. However, C++ (and possibly other libraries) do not by default provide a nice exported interface. In C++, because of features like overloading, the names are "mangled" in a compiler specific way. But you can manually export a nice interface:

 Let's try a slighly more advanced example, this time in C++:

In [ ]:
%%writefile templates.cpp


template<class T>
T square (T x) {
    return x*x;
}


extern "C" {
    int square_int(int x) {
        return square(x);
    }
    
    double square_double(double x) {
        return square(x);
    }
}

In [ ]:
!{CXX} templates.cpp -shared -o templates.so

In [ ]:
from functools import singledispatchmethod

class TemplateLib(object):
    def __init__(self, file_to_load: str):
        self._lib = ctypes.cdll.LoadLibrary(file_to_load)
        
        self._lib.square_double.argtypes = (ctypes.c_double,)
        self._lib.square_double.restype = ctypes.c_double
        
        self._lib.square_int.argtypes = (ctypes.c_int,)
        self._lib.square_int.restype = ctypes.c_int
        
        
    @singledispatchmethod
    def square(self, arg):
        raise NotImplementedError("Not a registered type")

    @square.register
    def _(self, x: int):
        return self._lib.square_int(x)

    @square.register
    def _(self, x: float):
        return self._lib.square_double(x)

In [ ]:
templates = TemplateLib('./templates.so')

In [ ]:
templates.square(2)

In [ ]:
templates.square(2.0)

## Numpy tools

Let's breifly mention Numpy has adaptors to help you with ctypes, called [numpy.ctypeslib](https://numpy.org/doc/stable/reference/routines.ctypeslib.html?highlight=ctypeslib#module-numpy.ctypeslib). You can convert to/from *any* object providing an `__array_interface__`, like numpy arrays. It has a loader with more consistant defaults across operating systems. You also have a special pointer that can do bounds checking and such for array arguments.

# [CFFI](http://cffi.readthedocs.io/en/latest/overview.html)

* The *C Foreign Function Interface* for Python
* C only
* Developed for PyPy, but available in CPython too

The same example as before:

In [ ]:
from cffi import FFI
ffi = FFI()

ffi.cdef("float square(float);")

C = ffi.dlopen('./simple.so')

C.square(2.0)

Notice we were able to give a C header this time; no futzing around with C function types by hand. 

### Try it yourself

Let's try it now. We'll take a few minutes to play with the code; we are also available to help with troubleshooting at this time. Double the elements in this array (for simplicity, do it in-place):

<!---

alib.squares.argtypes = (ctypes.POINTER(ctypes.c_float), ctypes.c_int)
alib.squares.restype = None

alib.squares(np.ctypeslib.as_ctypes(arr), arr.size)
--->

In [ ]:
%%writefile array_square.c

void squares(float* arr, int size) {
    for(int i=0; i<size; i++)
        arr[i] = arr[i]*arr[i];
}

In [ ]:
!{CC} array_square.c -shared -o array_square.so

In [ ]:
import numpy as np
import ctypes

alib = ctypes.cdll.LoadLibrary('./array_square.so')

In [ ]:
... # setup function

Remember to match the types! This is a 32 bit float, not a 64 bit double!

In [ ]:
arr = np.array([1,2,3,4,5], dtype=np.float32)

In [ ]:
... # Call function

In [ ]:
arr

# Mix and match with Numba and more

We can mix ctypes (or cffi) [with Numba](http://numba.pydata.org/numba-doc/0.15.1/interface_c.html), as well! Here's another way to do the above problem:

In [ ]:
import numpy as np

arr = np.random.randint(1, 10, size=10_000)

In [ ]:
import numba

In [ ]:
# Question: Why do I have to do this? Any thoughts?
csquare = lib.square

@numba.vectorize
def squares(x):
    return csquare(x)

In [ ]:
squares(arr)